In [3]:
import os
import pandas as pd
import inro.modeller as m
import inro.emme.database.emmebank as eb
mm = m.Modeller()
desktop = mm.desktop
de = desktop.data_explorer()

k_list = []
m_list = []

In [10]:
year = ['2016', '2029', '2039', '2049']
period = ['AM', 'MD', 'PM', 'EV', 'NT']
root_ws_folder = desktop.root_worksheet_folder()
import_fromdb = mm.tool('inro.emme.data.database.import_from_database')

k_list = []
m_list = []

for i in year:
    for j in period:
        for k in eb.Emmebank("\\tore-infs01\Models\Hwy697\Highway 697\Subarea_ASET_{}\{}{}\emmebank".format(i, i, j)).scenarios():
            k_list.append(int(k))

m_list = range(len(k_list))

c = 0
for i in year:
    for j in range(len(period)):
        p = '{}{}'.format(i, period[j])
        print(p, k_list[j])

        eb.Emmebank("\\tore-infs01\Models\Hwy697\Highway 697\Subarea_ASET_{}\{}{}\emmebank".format(i, i, period[j])).copy_scenario(k_list[c], m_list[c] + 101, copy_path_files = True, copy_strat_files = True, copy_db_files = True)
        eb.Emmebank("\\tore-infs01\Models\Hwy697\Highway 697\Subarea_ASET_{}\{}{}\emmebank".format(i, i, period[j])).dispose()

        # Import into current database
        fun_path = "\\tore-infs01\Models\Hwy697\Highway 697\Subarea_ASET_{}\{}{}\emmebank".format(i, i, period[j])
        import_fromdb(src_database = fun_path,
                     src_scenario_ids =  m_list[c] + 101)

        eb.Emmebank("\\tore-infs01\Models\Hwy697\Highway 697\Subarea_ASET_{}\{}{}\emmebank".format(i, i, period[j])).delete_scenario(m_list[c] + 101)
        eb.Emmebank("\\tore-infs01\Models\Hwy697\Highway 697\Subarea_ASET_{}\{}{}\emmebank".format(i, i, period[j])).dispose()
        print("done with " + str(k_list[j]) + " , " + " scen" + str(m_list[c] + 101))

        c+=1
        
desktop.app.App.refresh_data()

('2016', 'AM', Scenario(2))
('2016', 'MD', Scenario(3))
('2016', 'PM', Scenario(4))
('2016', 'EV', Scenario(5))
('2016', 'NT', Scenario(1))
('2029', 'AM', Scenario(2))
('2029', 'MD', Scenario(3))
('2029', 'PM', Scenario(4))
('2029', 'EV', Scenario(5))
('2029', 'NT', Scenario(1))
('2039', 'AM', Scenario(2))
('2039', 'MD', Scenario(3))
('2039', 'PM', Scenario(4))
('2039', 'EV', Scenario(5))
('2039', 'NT', Scenario(1))
('2049', 'AM', Scenario(2))
('2049', 'MD', Scenario(3))
('2049', 'PM', Scenario(4))
('2049', 'EV', Scenario(5))
('2049', 'NT', Scenario(1))
('2016AM', 2)
done with 2 ,  scen101
('2016MD', 3)
done with 3 ,  scen102
('2016PM', 4)
done with 4 ,  scen103
('2016EV', 5)
done with 5 ,  scen104
('2016NT', 1)
done with 1 ,  scen105
('2029AM', 2)
done with 2 ,  scen106
('2029MD', 3)
done with 3 ,  scen107
('2029PM', 4)
done with 4 ,  scen108
('2029EV', 5)
done with 5 ,  scen109
('2029NT', 1)
done with 1 ,  scen110
('2039AM', 2)
done with 2 ,  scen111
('2039MD', 3)
done with 3 ,  scen

In [12]:
processed_dir = r"\\tore-infs01\Models\Hwy697\Highway 697\csvs"

for scen in range(101, 121):
    
    emmebank = inro.modeller.Modeller().emmebank
    scenarioooo = emmebank.scenario(scen)
    de.replace_primary_scenario(scenarioooo)
    root_ws_folder = desktop.root_worksheet_folder()
    
    p = scenarioooo.id
    
    links_sheet = root_ws_folder.find_item(['AT_LinkVol'])
    links = links_sheet.open()
    links.save_as_data_table(name = 'ATLinks_forExport',overwrite = True)
    links.close()
    dts = desktop.project.data_tables()
    links_dt = dts.table('ATLinks_forExport')
    data = links_dt.get_data()
    output_file = os.path.join(processed_dir,'ATLinks_' + p + '.csv')
    data.export_to_csv(output_file, separator = ',')

print("it's done bro")

its done bro


In [13]:
ATLinkVols = pd.DataFrame()
processed_dir = r"\\tore-infs01\Models\Hwy697\Highway 697\csvs"
defs = pd.read_csv('f\period_defs.csv')

from os import listdir
from os.path import isfile, join

scens = [f for f in listdir(processed_dir) if isfile(join(processed_dir, f))]

for scen in scens:
    key = scen[8:12]
    
    links_ATVol = pd.read_csv(os.path.join(processed_dir,'ATLinks_'+ p +'.csv'))
    links_ATVol['key'] = key
    ATLinkVols = ATLinkVols.append(links_ATVol)

ATLinkVols.to_csv(os.path.join(processed_dir, 'ATLinkVols_allScens.csv'), index = False)

defs = pd.read_csv('\\tore-infs01\Models\Hwy697\Highway 697\period_defs.csv')
allpers = pd.read_csv('\\tore-infs01\Models\Hwy697\Highway 697\csvs\ATLinkVols_allScens.csv')
allpers.merge(defs, on = 'key').drop(labels = 'key', axis = 1).to_csv(os.path.join(processed_dir, 'ATLinkVols_allScens.csv'), index = False)

print("it's done bro")